In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
using LinearAlgebra, Plots
import ForwardDiff as FD
using Printf
using JLD2

  Activating project at `~/Documents/CMU-16-745/originals/HW1_S23`


In [2]:
A = [1.0; 2.0; 3]
@show typeof(A)
@show Diagonal(A)
@show ones(length(A))
@show map((x,y) -> x+y, A, 2 .* A)
@show abs.(A)
@show map((x,y) -> (x > 2 && x > 2) ? 1 : -1, A, 2 .* A)

typeof(A) = Vector{Float64}
Diagonal(A) = [1.0 0.0 0.0; 0.0 2.0 0.0; 0.0 0.0 3.0]
ones(length(A)) = [1.0, 1.0, 1.0]
map(((x, y)->begin
            #= In[2]:5 =#
            x + y
        end), A, 2 .* A) = [3.0, 6.0, 9.0]
abs.(A) = [1.0, 2.0, 3.0]
map(((x, y)->begin
            #= In[2]:7 =#
            if x > 2 && x > 2
                1
            else
                -1
            end
        end), A, 2 .* A) = [-1, -1, 1]


3-element Vector{Int64}:
 -1
 -1
  1

In [3]:
import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

N = 10000
x = rand(N)

thing1 = x -> map(x_ -> x_ > 0.5 ? 1 : 0, x)

@benchmark thing1(x) setup=(x=rand(N))

   Resolving package versions...
  No Changes to `~/Documents/CMU-16-745/originals/HW1_S23/Project.toml`
  No Changes to `~/Documents/CMU-16-745/originals/HW1_S23/Manifest.toml`


BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  3.169 μs … 67.775 μs  ┊ GC (min … max):  0.00% … 74.96%
 Time  (median):     4.536 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.104 μs ±  4.828 μs  ┊ GC (mean ± σ):  10.07% ±  9.62%

  ▂█▆                                                        ▁
  ████▅▆▇▅▁▁▁▁▁▁▁▁▁▁▄▆▁▄▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▅▆ █
  3.17 μs      Histogram: log(frequency) by time     45.6 μs <

 Memory estimate: 78.17 KiB, allocs estimate: 2.

In [4]:

function thingo(x::Vector)::Vector
    y = ones(length(x))
    y[x .< 0.5] .= 0.0
    return y
end

@benchmark thingo(x) setup=(x=rand(N))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.193 μs … 579.703 μs  ┊ GC (min … max): 0.00% … 90.43%
 Time  (median):     12.413 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.438 μs ±  20.703 μs  ┊ GC (mean ± σ):  6.83% ±  4.32%

                            ▁▄▄▆▇███▇▆▅▂▂                       
  ▂▁▁▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▄▄▅▆▇███████████████▇▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂ ▄
  10.2 μs         Histogram: frequency by time         14.3 μs <

 Memory estimate: 121.45 KiB, allocs estimate: 7.

In [5]:
using Test
@test thingo(x) == thing1(x)

Test Passed

In [6]:
@benchmark x'*x setup=(x=rand(N))

BenchmarkTools.Trial: 10000 samples with 104 evaluations.
 Range (min … max):  782.471 ns …  1.075 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     820.500 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   826.770 ns ± 19.980 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▃       ▃▃       ▂█▇▂      ▁▆▅▁       ▁▂         ▁▁         ▂
  ██▇▃▁▄▁▄▇███▅▆▅▃▆▅██████▆████████▇▇▇▇▇████▇█████████▆▅▇▆▇▇▅▆ █
  782 ns        Histogram: log(frequency) by time       897 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [7]:
@benchmark norm(x).^2 setup=(x=rand(N))

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.156 μs …  2.972 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.160 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.164 μs ± 27.740 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▇▅▃▁                                                     ▂
  ███████▇▇▆▆▆▅▅▄▄▁▃▃▁▄▃▃▄▄▁▄▄▅▄▅▅▄▄▁▄▁▃▁▁▃▁▃▃▁▁▁▁▁▃▁▁▁▄▁▃▆█ █
  2.16 μs      Histogram: log(frequency) by time     2.31 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [8]:
@benchmark norm(x)^2 setup=(x=rand(N))

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.156 μs …  2.679 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.160 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.164 μs ± 26.359 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆█▇▄▂▁                                                     ▂
  ████████▆▅▆▆▃▄▆▅▄▁▄▄▁▃▃▁▃▃▃▄▃▃▄▄▄▃▁▁▁▃▃▁▁▄▁▁▁▁▁▃▁▁▃▃▁▄▁▁▆█ █
  2.16 μs      Histogram: log(frequency) by time     2.31 μs <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [9]:
@test (norm(x).^2 .- x'*x) .< 1e-9
@test (norm(x)^2 - x'*x) .< 1e-9

Test Passed

# Q2 (20 pts): Augmented Lagrangian Quadratic Program Solver

## Part (A): QP Solver (10 pts)
Here we are going to use the augmented lagrangian method described [here in a video](https://www.youtube.com/watch?v=0x0JD5uO_ZQ), with [the corresponding pdf here](https://github.com/Optimal-Control-16-745/lecture-notebooks-2022/blob/main/misc/AL_tutorial.pdf) to solve the following problem:

$$\begin{align}
\min_x \quad & \frac{1}{2}x^TQx + q^Tx \\ 
\mbox{s.t.}\quad &  Ax -b = 0 \\ 
&  Gx - h \leq 0 
\end{align}$$
where the cost function is described by $Q \in \mathbb{R}^{n \times n}$, $q \in \mathbb{R}^n$, an equality constraint is described by $A \in \mathbb{R}^{m \times n}$ and $b \in \mathbb{R}^m$, and an inequality constraint is described by $G \in \mathbb{R}^{p \times n}$ and $h \in \mathbb{R}^p$.


By introducing a dual variable $\lambda \in \mathbb{R}^m$ for the equality constraint, and $\mu \in \mathbb{R}^p$ for the inequality constraint, we have the following KKT conditions for optimality:

$$\begin{align}
Qx + q + A^T\lambda + G^T \mu &= 0 \quad \quad \text{stationarity}\\ 
Ax-b&= 0 \quad \quad \text{primal feasibility} \\ 
Gx-h&\leq 0 \quad \quad \text{primal feasibility} \\ 
\mu &\geq 0 \quad \quad \text{dual feasibility} \\ 
\mu \circ (Gx - h) &= 0 \quad \quad \text{complementarity}
  \end{align}$$
  where $\circ$ is element-wise multiplication.  

In [10]:
# TODO: read below
# NOTE: DO NOT USE A WHILE LOOP ANYWHERE
"""
The data for the QP is stored in `qp` the following way:
    @load joinpath(@__DIR__, "qp_data.jld2") qp 

which is a NamedTuple, where
    Q, q, A, b, G, h = qp.Q, qp.q, qp.A, qp.b, qp.G, qp.h

contains all of the problem data you will need for the QP.

Your job is to make the following function 
    
    x, λ, μ = solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-8)

You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:
You can use (or not use) any of the additional functions:

as long as solve_qp works. 
"""
function cost(qp::NamedTuple, x::Vector)::Real
    0.5*x'*qp.Q*x + dot(qp.q,x)
end
function c_eq(qp::NamedTuple, x::Vector)::Vector
    qp.A*x - qp.b 
end
function h_ineq(qp::NamedTuple, x::Vector)::Vector
    qp.G*x - qp.h
end

function mask_matrix(qp::NamedTuple, x::Vector, μ::Vector, ρ::Real)::Matrix
    return Diagonal(map( (h_,μ_) -> (h_ .< 0 && μ_ == 0.0) ? 0.0 : ρ, h_ineq(qp, x), μ) )
end
function augmented_lagrangian(qp::NamedTuple, x::Vector, λ::Vector, μ::Vector, ρ::Real)::Real
    c = c_eq(qp, x)
    h = h_ineq(qp, x)
    L = cost(qp, x) + λ' * c + μ' * h
    Iₚ = mask_matrix(qp, x, μ, ρ)
    return L + (ρ/2.0) * (c' * c) + 0.5 * (h' * Iₚ * h)
end
function logging(qp::NamedTuple, main_iter::Int, AL_gradient::Vector, x::Vector, λ::Vector, μ::Vector, ρ::Real)
    # TODO: stationarity norm
    ∇f = qp.Q * x + qp.q #FD.gradient(x_ -> cost(qp,x_), x)
    ∂c = qp.A' * λ #FD.jacobian(x_ -> c_eq(qp, x_), x) * λ
    ∂h = qp.G' * μ #FD.jacobian(x_ -> h_ineq(qp, x_), x) * μ
    stationarity_norm = norm( ∇f + ∂c + ∂h ) # fill this in 
    @printf("%3d  % 7.2e  % 7.2e  % 7.2e  % 7.2e  % 7.2e  %5.0e\n",
          main_iter, stationarity_norm, norm(AL_gradient), maximum(h_ineq(qp,x)),
          norm(c_eq(qp,x),Inf), abs(dot(μ,h_ineq(qp,x))), ρ)
end
function solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-8)
    x = zeros(length(qp.q))
    λ = zeros(length(qp.b))
    μ = zeros(length(qp.h))
    ρ = 1.0
    ϕ = 10.0
    newton_tol = 1e-6
    
    if verbose
        @printf "iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ\n"
        @printf "----------------------------------------------------------------\n"
    end
    
    # TODO:
    for main_iter = 1:max_iters 
        if verbose
            logging(qp, main_iter, zeros(1), x, λ, μ, 0.0)
        end
        
        # NOTE: when you do your dual update for μ, you should compute
        # your element-wise maximum with `max.(a,b)`, not `max(a,b)`
        
        # 1. solve min_x using newtons, for a qp this is boils down to the hessian
        # x = newtons_method( x_ -> augmented_lagrangian(qp, x_, λ, μ, ρ), x )
        for i = 1:max_iters
            H = FD.hessian(x_ -> augmented_lagrangian(qp, x_, λ, μ, ρ), x)
            g = FD.gradient( x_ -> augmented_lagrangian(qp, x_, λ, μ, ρ), x)
            x += -H\g # No regularisation
            if norm(g) < newton_tol
                break
            end
        end
        
        
        # 2. update dual variables
        cx = c_eq(qp, x)
        hx = h_ineq(qp, x)
        λ += ρ * cx
        μ = max.(0.0, μ + ρ * hx)
        
        # 3. update penalty 
        ρ *= ϕ
        
        # 4. Check convergence against KKT conditions
        # Stationarity 
        ∇f = qp.Q * x + qp.q #FD.gradient(x_ -> cost(qp,x_), x)
        ∂c = qp.A' * λ #FD.jacobian(x_ -> c_eq(qp, x_), x) * λ
        ∂h = qp.G' * μ #
        ϵ = norm( [∇f + ∂c + ∂h; cx; μ .* hx] )
        
        # TODO: convergence criteria based on tol 
        if ϵ < tol
            return x, λ, μ
        end
    end
    error("qp solver did not converge")
end
let 
    # example solving qp 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp; verbose = true, tol = 1e-8)
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   2.98e+01   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  2   6.19e-15   0.00e+00   5.51e-01   1.27e+00   4.59e-01  0e+00
  3   6.16e+00   0.00e+00   2.56e-02   3.07e-01   1.05e-02  0e+00
  4   5.52e-01   0.00e+00   6.84e-03   1.35e-02   7.94e-03  0e+00
  5   2.71e-12   0.00e+00   3.64e-05   1.62e-04   1.06e-04  0e+00
  6   2.13e-11   0.00e+00  -5.61e-09   2.05e-08   1.14e-08  0e+00


([-0.32623080571340296, 0.24943797997188885, -0.4322676644050712, -1.4172246971241347, -1.3994527400876549, 0.6099582408523684, -0.07312202122159445, 1.3031477521999635, 0.5389034791065493, -0.7225813651685946], [-0.12835195126257304, -2.837624167273024, -0.8320804499706329], [0.03635294260329047, 0.0, 0.0, 1.0594444951106212, 0.0])

### QP Solver test

In [11]:
# 10 points 
using Test 
@testset "qp solver" begin 
    @load joinpath(@__DIR__, "qp_data.jld2") qp 
    x, λ, μ = solve_qp(qp; verbose = true, max_iters = 100, tol = 1e-6)
    
    @load joinpath(@__DIR__, "qp_solutions.jld2") qp_solutions
    @test norm(x - qp_solutions.x,Inf)<1e-3;
    @test norm(λ - qp_solutions.λ,Inf)<1e-3;
    @test norm(μ - qp_solutions.μ,Inf)<1e-3;
end

iter   |∇Lₓ|      |∇ALₓ|     max(h)     |c|        compl     ρ
----------------------------------------------------------------
  1   2.98e+01   0.00e+00   4.38e+00   6.49e+00   0.00e+00  0e+00
  2   6.19e-15   0.00e+00   5.51e-01   1.27e+00   4.59e-01  0e+00
  3   6.16e+00   0.00e+00   2.56e-02   3.07e-01   1.05e-02  0e+00
  4   5.52e-01   0.00e+00   6.84e-03   1.35e-02   7.94e-03  0e+00
  5   2.71e-12   0.00e+00   3.64e-05   1.62e-04   1.06e-04  0e+00
Test Summary: | Pass  Total  Time
qp solver     |    3      3  0.2s


Test.DefaultTestSet("qp solver", Any[], 3, false, false, true, 1.694471120655473e9, 1.694471120890463e9, false)

# Simulating a Falling Brick with QPs
In this question we'll be simulating a brick falling and sliding on ice in 2D. You will show that this problem can be formulated as a QP, which you will solve using an Augmented Lagrangian method.

## The Dynamics
The dynamics of the brick can be written in continuous time as
$$ M \dot{v}  + M g = J^T \lambda \\ \text{ where } M = mI_{2\times 2}, \; g = \begin{bmatrix} 0 \\ 9.81 \end{bmatrix},\; J = \begin{bmatrix} 0 & 1 \end{bmatrix} $$
and $\lambda \in \mathbb{R}$ is the normal force. The velocity $v \in \mathbb{R}^2$ and position $q \in \mathbb{R}^2$ are composed of the horizontal and vertical components.

We can discretize the dynamics with backward Euler:
$$ \begin{bmatrix} v_{k+1} \\ q_{k+1} \end{bmatrix} = \begin{bmatrix} v_k \\ q_k \end{bmatrix}
+ \Delta t \cdot \begin{bmatrix} \frac{1}{m} J^T \lambda_{k+1} - g \\ v_{k+1} \end{bmatrix}$$

We also have the following contact constraints:
$$ \begin{align}
J q_{k+1} &\geq 0 &&\text{(don't fall through the ice)} \\
\lambda_{k+1} &\geq 0 &&\text{(normal forces only push, not pull)} \\
\lambda_{k+1} J q_{k+1} &= 0 &&\text{(no force at a distance)}
\end{align} $$

## Part (B): QP formulation for Falling Brick (5 pts)
Show that these discrete-time dynamics are equivalent to the following QP by writing down the KKT conditions.

$$ \begin{align}
    &\text{minimize}_{v_{k+1}} && \frac{1}{2} v_{k+1}^T M v_{k+1} + [M (\Delta t \cdot g - v_k)]^Tv_{k+1} \\
    &\text{subject to} && -J(q_k + \Delta t \cdot v_{k+1}) \leq 0 \\
\end{align} $$

**TASK**: Write down the KKT conditions for the optimization problem above, and show that it's equivalent to the dynamics problem stated previously. Use LaTeX markdown.

**PUT ANSWER HERE:**

## Part (C): Brick Simulation (5 pts)

In [12]:
function brick_simulation_qp(q, v; mass = 1.0, Δt = 0.01)
    
    # TODO: fill in the QP problem data for a simulation step 
    # fill in Q, q, G, h, but leave A, b the same 
    # this is because there are no equality constraints in this qp 
    
    qp = (
        Q = zeros(2,2), 
        q = zeros(2),
        A = zeros(0,2), # don't edit this
        b = zeros(0),   # don't edit this 
        G = zeros(1,2),
        h = zeros(1)
    )
    
    return qp 
end

brick_simulation_qp (generic function with 1 method)

In [13]:
@testset "brick qp" begin 
    
    q = [1,3.0]
    v = [2,-3.0]
    
    qp = brick_simulation_qp(q,v)
    
    # check all the types to make sure they're right
    qp.Q::Matrix{Float64}
    qp.q::Vector{Float64}
    qp.A::Matrix{Float64}
    qp.b::Vector{Float64}
    qp.G::Matrix{Float64}
    qp.h::Vector{Float64}
    
    @test size(qp.Q) == (2,2)
    @test size(qp.q) == (2,)
    @test size(qp.A) == (0,2)
    @test size(qp.b) == (0,)
    @test size(qp.G) == (1,2)
    @test size(qp.h) == (1,)
    
    @test abs(tr(qp.Q) - 2) < 1e-10
    @test norm(qp.q - [-2.0, 3.0981]) < 1e-10 
    @test norm(qp.G - [0 -.01]) < 1e-10 
    @test abs(qp.h[1] -3) < 1e-10
    
end

brick qp: Test Failed at In[13]:23
  Expression: abs(tr(qp.Q) - 2) < 1.0e-10
   Evaluated: 2.0 < 1.0e-10

Stacktrace:
 [1] macro expansion
   @ /opt/julia-1.9.2/share/julia/stdlib/v1.9/Test/src/Test.jl:478 [inlined]
 [2] macro expansion
   @ In[13]:23 [inlined]
 [3] macro expansion
   @ /opt/julia-1.9.2/share/julia/stdlib/v1.9/Test/src/Test.jl:1498 [inlined]
 [4] top-level scope
   @ In[13]:3
brick qp: Test Failed at In[13]:24
  Expression: norm(qp.q - [-2.0, 3.0981]) < 1.0e-10
   Evaluated: 3.6875769293670335 < 1.0e-10

Stacktrace:
 [1] macro expansion
   @ /opt/julia-1.9.2/share/julia/stdlib/v1.9/Test/src/Test.jl:478 [inlined]
 [2] macro expansion
   @ In[13]:24 [inlined]
 [3] macro expansion
   @ /opt/julia-1.9.2/share/julia/stdlib/v1.9/Test/src/Test.jl:1498 [inlined]
 [4] top-level scope
   @ In[13]:3
brick qp: Test Failed at In[13]:25
  Expression: norm(qp.G - [0 -0.01]) < 1.0e-10
   Evaluated: 0.01 < 1.0e-10

Stacktrace:
 [1] macro expansion
   @ /opt/julia-1.9.2/share/julia/stdl

LoadError: [91mSome tests did not pass: 6 passed, 4 failed, 0 errored, 0 broken.[39m

In [14]:
include(joinpath(@__DIR__, "animate_brick.jl"))
let 
    
    dt = 0.01 
    T = 3.0 
    
    t_vec = 0:dt:T
    N = length(t_vec)
    
    qs = [zeros(2) for i = 1:N]
    vs = [zeros(2) for i = 1:N]
    
    qs[1] = [0, 1.0]
    vs[1] = [1, 4.5]
    
    # TODO: simulate the brick by forming and solving a qp 
    # at each timestep. Your QP should solve for vs[k+1], and
    # you should use this to update qs[k+1]

    
    xs = [q[1] for q in qs]
    ys = [q[2] for q in qs]
    
    @show @test abs(maximum(ys)-2)<1e-1
    @show @test minimum(ys) > -1e-2
    @show @test abs(xs[end] - 3) < 1e-2
    
    xdot = diff(xs)/dt
    @show @test maximum(xdot) < 1.0001
    @show @test minimum(xdot) > 0.9999
    @show @test ys[110] > 1e-2
    @show @test abs(ys[111]) < 1e-2
    @show @test abs(ys[112]) < 1e-2
    
    display(plot(xs, ys, ylabel = "y (m)", xlabel = "x (m)"))
    
    animate_brick(qs)
    
    
    
end

Test Failed at In[14]:24
  Expression: abs(maximum(ys) - 2) < 0.1
   Evaluated: 1.0 < 0.1



LoadError: [91mThere was an error during testing[39m